In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
from datasets.endovis import Endovis2017
import numpy as np
import torch
import matplotlib.pyplot as plt
import inspect
from general_utils import TrainingLogger, get_attribute, filter_args, log, training_config_from_cli_args
from general_utils import AttributeDict

In [69]:
import yaml

In [89]:
experiment_name = "endovis2017.yaml"
experiment_id = 0

yaml_config = yaml.load(open(f'experiments/{experiment_name}'), Loader=yaml.SafeLoader) 

config = yaml_config['configuration']

config = {**config, **yaml_config['individual_configurations'][experiment_id]}

config = AttributeDict(config) #a new type


In [90]:
dataset_cls = get_attribute(config.dataset)  #😉 dataset args from config
_, dataset_args, _ = filter_args(config, inspect.signature(dataset_cls).parameters) #😉 dataset args filtered

In [91]:
dataset_args

{'split': 'train',
 'mask': 'text_and_crop_blur_highlight352',
 'image_size': 352,
 'aug': '1new',
 'with_visual': True,
 'negative_prob': 0.2}

In [92]:
training_data = Endovis2017(**dataset_args)

In [93]:
len(training_data)

12281

In [94]:
from PIL import Image

In [95]:
a.shape

(1024, 1280, 3)

In [96]:
training_data[0]

cropped_train\instrument_dataset_1\clipseg_masks\frame000_bipolar_forceps.png
(1024, 1280, 3)
(1024, 1280)
torch.Size([3, 352, 352])
torch.Size([352, 352])
./datasets/Endovis2017/visual_prompt\no_combination\instrument_segmentation\img\bipolar_forceps.jpg
./datasets/Endovis2017/visual_prompt\no_combination\instrument_segmentation\mask\bipolar_forceps.jpg
(339, 512, 3)
(339, 512, 3)


RuntimeError: result type Float can't be cast to the desired output type Byte

In [27]:


figure = plt.figure(figsize=(8, 8))
num_of_samples = 5
num_img_per_sample = 5
rows, cols = num_of_samples, num_img_per_sample #😉 4 because I want to display img1, img2, semantic, opticalflow
for i in range(0, num_of_samples):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label_map, flow_map, _ = training_data[sample_idx]

    # print(img.shape)
    # print(label_map.shape)
    # print(flow_map.shape)

    img = np.moveaxis(img.numpy(), 0, -1)
    img = img.astype(np.uint8)
    img1 = img[...,0:3]
    img2 = img[...,3:6]


    label_map = label_map.numpy()
    sem1_color_img = transform_label_map_to_colour_segmentation(label_map[0,...])
    sem2_color_img = transform_label_map_to_colour_segmentation(label_map[1,...])

    
    figure.add_subplot(rows, cols, (i*num_img_per_sample) + 1 )
    plt.title(f"{sample_idx}_img1")
    plt.axis("off")
    plt.imshow(img1)

    figure.add_subplot(rows, cols, (i*num_img_per_sample) + 2)
    plt.title(f"{sample_idx}_img2")
    plt.axis("off")
    plt.imshow(img2)

    figure.add_subplot(rows, cols, (i*num_img_per_sample) + 3)
    plt.title(f"{sample_idx}_sem1")
    plt.axis("off")
    plt.imshow(sem1_color_img)

    figure.add_subplot(rows, cols, (i*num_img_per_sample) + 4)
    plt.title(f"{sample_idx}_sem2")
    plt.axis("off")
    plt.imshow(sem2_color_img)

    figure.add_subplot(rows, cols, (i*num_img_per_sample) + 5)
    plt.title(f"{sample_idx}_flow")
    plt.axis("off")
    plt.imshow(flow_map_img)
plt.show()

TypeError: __init__() got an unexpected keyword argument 'train'